In [21]:
import torch, torchvision
from mobilenetv3 import MobileNetV3_forFPN, MobileNetV3, load_pretrained_fpn
from tests import test_loaded_weights, compare_output
%load_ext autoreload
%autoreload 2
#torch.backends.cudnn.deterministic = True

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
mn3_fpn = MobileNetV3_forFPN()
mn3_fpn = load_pretrained_fpn(mn3_fpn,\
                              '/home/davidyuk/Projects/backbones/pytorch-mobilenet-v3/mobilenetv3_small_67.4.pth.tar')
with torch.no_grad():
    print(mn3_fpn.forward(torch.zeros(1,3,224,224)).size())
# tuple(mn3_fpn.parameters())

torch.Size([1, 96, 7, 7])


In [5]:
test_loaded_weights()

test passed


In [7]:
for name, parameter in mn3_fpn.named_parameters():
    print(name)

layers_os4.0.0.weight
layers_os4.0.1.weight
layers_os4.0.1.bias
layers_os4.1.conv.0.weight
layers_os4.1.conv.1.weight
layers_os4.1.conv.1.bias
layers_os4.1.conv.3.weight
layers_os4.1.conv.4.weight
layers_os4.1.conv.4.bias
layers_os4.1.conv.5.fc.0.weight
layers_os4.1.conv.5.fc.2.weight
layers_os4.1.conv.7.weight
layers_os4.1.conv.8.weight
layers_os4.1.conv.8.bias
layers_os8.0.conv.0.weight
layers_os8.0.conv.1.weight
layers_os8.0.conv.1.bias
layers_os8.0.conv.3.weight
layers_os8.0.conv.4.weight
layers_os8.0.conv.4.bias
layers_os8.0.conv.7.weight
layers_os8.0.conv.8.weight
layers_os8.0.conv.8.bias
layers_os8.1.conv.0.weight
layers_os8.1.conv.1.weight
layers_os8.1.conv.1.bias
layers_os8.1.conv.3.weight
layers_os8.1.conv.4.weight
layers_os8.1.conv.4.bias
layers_os8.1.conv.7.weight
layers_os8.1.conv.8.weight
layers_os8.1.conv.8.bias
layers_os16.0.conv.0.weight
layers_os16.0.conv.1.weight
layers_os16.0.conv.1.bias
layers_os16.0.conv.3.weight
layers_os16.0.conv.4.weight
layers_os16.0.conv.4.bi

In [33]:
from torchvision.models.detection.backbone_utils import BackboneWithFPN

def mobilenetV3_fpn_backbone(pretrained=False):
    backbone = MobileNetV3_forFPN()
    
    if pretrained:
        load_pretrained_fpn(backbone,\
                              '/home/davidyuk/Projects/backbones/pytorch-mobilenet-v3/mobilenetv3_small_67.4.pth.tar')
        
    return_layers = {'layers_os4': 0, 'layers_os8': 1, 'layers_os16': 2, 'layers_os32': 3}

    in_channels_list = [16, 24, 48, 96]
    out_channels = 100
    return BackboneWithFPN(backbone, return_layers, in_channels_list, out_channels)

In [36]:
mb3_ = mobilenetV3_fpn_backbone(pretrained=True)
mb3_.forward(torch.ones(1,3,224,224))[0].size()

torch.Size([1, 100, 56, 56])

In [ ]:
backbone (nn.Module): the network used to compute the features for the model.
    It should contain a out_channels attribute, which indicates the number of output
    channels that each feature map has (and it should be the same for all feature maps).
    The backbone should return a single Tensor or and OrderedDict[Tensor].
num_classes (int): number of output classes of the model (including the background).
    If box_predictor is specified, num_classes should be None.
min_size (int): minimum size of the image to be rescaled before feeding it to the backbone
max_size (int): maximum size of the image to be rescaled before feeding it to the backbone
image_mean (Tuple[float, float, float]): mean values used for input normalization.
    They are generally the mean values of the dataset on which the backbone has been trained
    on
image_std (Tuple[float, float, float]): std values used for input normalization.
    They are generally the std values of the dataset on which the backbone has been trained on
rpn_anchor_generator (AnchorGenerator): module that generates the anchors for a set of feature
    maps.
rpn_head (nn.Module): module that computes the objectness and regression deltas from the RPN
rpn_pre_nms_top_n_train (int): number of proposals to keep before applying NMS during training
rpn_pre_nms_top_n_test (int): number of proposals to keep before applying NMS during testing
rpn_post_nms_top_n_train (int): number of proposals to keep after applying NMS during training
rpn_post_nms_top_n_test (int): number of proposals to keep after applying NMS during testing
rpn_nms_thresh (float): NMS threshold used for postprocessing the RPN proposals
rpn_fg_iou_thresh (float): minimum IoU between the anchor and the GT box so that they can be
    considered as positive during training of the RPN.
rpn_bg_iou_thresh (float): maximum IoU between the anchor and the GT box so that they can be
    considered as negative during training of the RPN.
rpn_batch_size_per_image (int): number of anchors that are sampled during training of the RPN
    for computing the loss
rpn_positive_fraction (float): proportion of positive anchors in a mini-batch during training
    of the RPN
box_roi_pool (MultiScaleRoIAlign): the module which crops and resizes the feature maps in
    the locations indicated by the bounding boxes
box_head (nn.Module): module that takes the cropped feature maps as input
box_predictor (nn.Module): module that takes the output of box_head and returns the
    classification logits and box regression deltas.
box_score_thresh (float): during inference, only return proposals with a classification score
    greater than box_score_thresh
box_nms_thresh (float): NMS threshold for the prediction head. Used during inference
box_detections_per_img (int): maximum number of detections per image, for all classes.
box_fg_iou_thresh (float): minimum IoU between the proposals and the GT box so that they can be
    considered as positive during training of the classification head
box_bg_iou_thresh (float): maximum IoU between the proposals and the GT box so that they can be
    considered as negative during training of the classification head
box_batch_size_per_image (int): number of proposals that are sampled during training of the
    classification head
box_positive_fraction (float): proportion of positive proposals in a mini-batch during training
    of the classification head
bbox_reg_weights (Tuple[float, float, float, float]): weights for the encoding/decoding of the
    bounding boxes
mask_roi_pool (MultiScaleRoIAlign): the module which crops and resizes the feature maps in
     the locations indicated by the bounding boxes, which will be used for the mask head.
mask_head (nn.Module): module that takes the cropped feature maps as input
mask_predictor (nn.Module): module that takes the output of the mask_head and returns the
    segmentation mask logits

In [36]:
%load_ext autoreload
%autoreload 0
import torch, torchvision
from MASKRCNN_creator import maskrcnn_mobileNetV3_fpn

chkp_path = '/home/davidyuk/Projects/backbones/pytorch-mobilenet-v3/mobilenetv3_small_67.4.pth.tar'
# model = maskrcnn_mobileNetV3_fpn(num_classes=3, backbone_chkp=chkp_path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# FeatMap-size-aware heads and necks

In [33]:
out_channels = 100#backbone.out_channels

from torchvision.models.detection.rpn import AnchorGenerator
anchor_sizes = ((32,), (64,), (128,), (256,), (512,))
aspect_ratios = ((1.0,),) * len(anchor_sizes)
rpn_anchor_generator = AnchorGenerator(
    anchor_sizes, aspect_ratios
)
# anchor_sizes = ((1,),(1,),(1,),(1,))#((50, 100), (25, 50), (40,15), (14,))#tuple(reversed())
# aspect_ratios = ((1.0,),) * len(anchor_sizes)
# rpn_anchor_generator = AnchorGenerator(
#     anchor_sizes, aspect_ratios
# )

# from torchvision.models.detection.rpn import RPNHead
# rpn_head = RPNHead(
#     out_channels, rpn_anchor_generator.num_anchors_per_location()[0]
# )

# rpn_pre_nms_top_n = dict(training=rpn_pre_nms_top_n_train, testing=rpn_pre_nms_top_n_test)
# rpn_post_nms_top_n = dict(training=rpn_post_nms_top_n_train, testing=rpn_post_nms_top_n_test)

# from torchvision.models.detection.rpn import RegionProposalNetwork
# rpn = RegionProposalNetwork(
#     rpn_anchor_generator, rpn_head,
#     rpn_fg_iou_thresh, rpn_bg_iou_thresh,
#     rpn_batch_size_per_image, rpn_positive_fraction,
#     rpn_pre_nms_top_n, rpn_post_nms_top_n, rpn_nms_thresh)

In [26]:
aspect_ratios

((1.0,), (1.0,), (1.0,), (1.0,), (1.0,))

In [20]:
rpn_anchor_generator.num_anchors_per_location()

[1, 1, 1, 1]

### Faster RCNN heads

In [2]:
from torchvision.ops import MultiScaleRoIAlign
box_roi_pool = MultiScaleRoIAlign(
    featmap_names=[0, 1, 2, 3],
    output_size=7,
    sampling_ratio=2)

# from torchvision.models.detection.faster_rcnn import TwoMLPHead
# resolution = box_roi_pool.output_size[0]
# representation_size = 1024
# box_head = TwoMLPHead(
#     out_channels * resolution ** 2,
#     representation_size)

# from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
# representation_size = 1024
# box_predictor = FastRCNNPredictor(
#     representation_size,
#     num_classes)

# from torchvision.models.detection.roi_heads import RoIHeads
# roi_heads = RoIHeads(
#     # Box
#     box_roi_pool, box_head, box_predictor,
#     box_fg_iou_thresh, box_bg_iou_thresh,
#     box_batch_size_per_image, box_positive_fraction,
#     bbox_reg_weights,
#     box_score_thresh, box_nms_thresh, box_detections_per_img)

### MASK RCNN heads

In [3]:
from torchvision.ops import MultiScaleRoIAlign
mask_roi_pool = MultiScaleRoIAlign(
    featmap_names=[0, 1, 2, 3],
    output_size=14,
    sampling_ratio=2)

# from torchvision.models.detection.mask_rcnn import MaskRCNNHeads
# mask_layers = (256, 256, 256, 256)
# mask_dilation = 1
# mask_head = MaskRCNNHeads(out_channels, mask_layers, mask_dilation)

# from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
# mask_predictor_in_channels = 256  # == mask_layers[-1]
# mask_dim_reduced = 256
# mask_predictor = MaskRCNNPredictor(mask_predictor_in_channels,
#                                    mask_dim_reduced, num_classes)

### Keypoints RCNN heads

## Putting everything together

In [37]:
kwargs = {'mask_roi_pool' : mask_roi_pool,
            'box_roi_pool' : box_roi_pool}                      
#           'rpn_anchor_generator' : rpn_anchor_generator}
#           'rpn_head':rpn_head}
model = maskrcnn_mobileNetV3_fpn(num_classes=3, backbone_chkp=chkp_path, **kwargs)
model.eval()
inp = torch.ones(1,3,800,800)
with torch.no_grad():
    %time out = model.forward(inp)
out

CPU times: user 7.21 s, sys: 149 ms, total: 7.36 s
Wall time: 1.12 s


[{'boxes': tensor([[  0.0000, 562.9419,   0.0000, 563.2704],
          [  0.0000, 318.9420,   0.0000, 319.2704],
          [  0.0000, 326.9419,   0.0000, 327.2704],
          [  0.0000, 314.9419,   0.0000, 315.2704],
          [  0.0000, 354.9420,   0.0000, 355.2705],
          [  0.0000, 338.9420,   0.0000, 339.2704],
          [  0.0000, 322.9419,   0.0000, 323.2704],
          [  0.0000, 578.9419,   0.0000, 579.2704],
          [  0.0000, 346.9420,   0.0000, 347.2705],
          [  0.0000, 358.9420,   0.0000, 359.2705],
          [  0.0000, 350.9419,   0.0000, 351.2704],
          [  0.0000, 330.9419,   0.0000, 331.2704],
          [  0.0000, 398.9419,   0.0000, 399.2704],
          [  0.0000, 574.9419,   0.0000, 575.2704],
          [  0.0000, 246.9417,   0.0000, 247.2702],
          [  0.0000, 250.9418,   0.0000, 251.2703],
          [  0.0000, 682.9642,   0.0000, 683.2930],
          [  0.0000, 214.9451,   0.0000, 215.2736],
          [  0.0000, 258.9418,   0.0000, 259.2703],
   

In [40]:
with torch.no_grad():
    %time out = model.forward(inp)

CPU times: user 7.45 s, sys: 210 ms, total: 7.66 s
Wall time: 1.15 s


In [27]:
model

MaskRCNN(
  (transform): GeneralizedRCNNTransform()
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (layers_os4): Sequential(
        (0): Sequential(
          (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): Hswish()
        )
        (1): MobileBottleneck(
          (conv): Sequential(
            (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU(inplace)
            (3): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
            (4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (5): SEModule(
              (avg_pool): AdaptiveAvgPool2d(output_size=1)
              (fc): Sequential(
             

In [31]:
from torchvision.models.detection.mask_rcnn import maskrcnn_resnet50_fpn
model_stock = maskrcnn_resnet50_fpn('resnet')
model_stock

MaskRCNN(
  (transform): GeneralizedRCNNTransform()
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d()
      (relu): ReLU(inplace)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d()
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d()
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d()
          (relu): ReLU(inplace)
          (downsample): Sequential(
            (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): FrozenBatchNorm2d()
          )
        )
    

In [39]:
from torchvision.models.detection.backbone_utils import resnet_fpn_backbone
resnet50_fpn = resnet_fpn_backbone('resnet50', False)
resnet50_fpn.eval()
inp = torch.ones(1,3,800,800)
with torch.no_grad():
    %time out = resnet50_fpn.forward(inp)


CPU times: user 8.38 s, sys: 676 ms, total: 9.06 s
Wall time: 854 ms


In [32]:
len(out)

5